## Deterministic Column Subset Selection

In [463]:
# Lifted from https://github.com/srmcc/dcss_single_cell/blob/master/src/dcss/dls_funct.py

import pandas as pd
import numpy as np
import scipy as scp

# Something is likely missing here, this can't be all there is to it.
# The function supposedDCSS incorporates this function below, but I'm
# still not 100% sure it's correct usage
def det_leverage(V, k, epsilon):
    """
    for the data matrix A=U \Sigma V^T
    A.shape =(n=number of samples, d=number of features)
    V.shape=(d, n)
    k is the rank of the PCA leverage score
    epsilon is the error parameter.
    the function returns
    theta: the number of kept columns
    index_keep: the index of the selected columns
    tau_sorted:  the sorted leverage scores of all the columns
    index_drop:  the index of the dropped columns
    """
    V = pd.DataFrame(V)
    Vk= V.iloc[:, 0:k]
    #print(Vk.shape)
    tau= np.sum(Vk**2, axis=1)
    tau_sorted= tau.sort_values(ascending=False, inplace=False)
    lev_sum=0
    for i in range(V.shape[0]):
        lev_sum =lev_sum+ tau_sorted.iloc[i]
        if lev_sum > k - epsilon:
            theta=i+1
            if theta >= k:
                break
    index_keep = tau_sorted.index[0:theta]
    index_keep = index_keep.values
    index_drop = tau_sorted.index[theta:]
    index_drop = index_drop.values
    return(theta, index_keep, tau_sorted, index_drop)

# Reads a path to .tsv file, returns the bitmatrix, 
# indexes pointing to fasta files (sort of redundant)
# and clique member names in order relative to bitmatrix
def read_bitmatrix(path):
    tsv_file = pd.read_csv(path, sep='\t').sort_values('query_name')
    bitmat = tsv_file.values[:,1:]
    index = tsv_file.values[:,0]
    members = list(tsv_file.columns[1:])
    return(bitmat,index,members)

# An attempt to use the DCSS method from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6347249/
# It's probably missing some vital steps...
def supposedDCSS(TCC,k,epsilon):
    TCC_svd = scp.sparse.linalg.svds(TCC, k)
    #TCC_svd = scp.sparse.linalg.svds(scp.sparse.csr_matrix(TCC), k)
    V=pd.DataFrame(TCC_svd[2].T)

    theta_TCC, index_keep_TCC, tau_sorted, index_drop = det_leverage(V, k, epsilon)
    return(TCC[:,index_keep_TCC], index_keep_TCC, index_drop)

# From a bitmatrix, get a submatrix only consisting of unique
# vector forms founds in the original matrix
# Returns a reduced matrix consisting of every variation of 
# row vector, i.e. every way a node in CCDBG can be colored 
# differently, as well as a reference list which maps the 
# row index of a vector in the reduced matrix to the indices of
# that vector form's corresponding fasta sequences.
# Also returns members, a list of member names in order
# of appearance in bitmatrix columns.
def reduceBitMatrix(path):
    bitmat, index, members = read_bitmatrix(path)
    uniques = set()
    mapping = dict()
    for i in range(bitmat.shape[0]):
        if (bitmat[i] == 0).any():
            form = tuple(bitmat[i])
            uniques.add(form)
            if form in mapping:
                mapping[form].append(index[i])
            else:
                mapping[form] = [index[i]]
    N = len(uniques)
    M = len(members)
    reduced = np.zeros((N,M))
    ref_to_fasta = list()
    for i,form in enumerate(uniques):
            reduced[i] = np.array(form)
            ref_to_fasta.append(mapping[form])
    return(reduced, ref_to_fasta, members)

In [452]:
def calculate_rank_k_subspace_leverage_score(A, i, k):
    
    a_i = A[:,i]
    u, s, vh = np.linalg.svd(A, full_matrices=False)
    U_k = u[:,:k]
    S_k = np.diag(s[:k])
    Vh_k = vh[:k]
    
    # rank_k_approximation
    M = np.dot(U_k, np.dot(S_k, Vh_k))
    
    # Moore-Penrose pseudoinverse
    MP_inv = np.linalg.pinv(np.matmul(M,M.T))
    
    ith_leverage_score = np.matmul(a_i.T,np.matmul(MP_inv,a_i))
    return(ith_leverage_score)

def homemade_DCSS(A, k, epsilon):
    n = A.shape[0]
    d = A.shape[1]
    leverage_scores = np.zeros(d)
    for i in range(d):
        leverage_scores[i] = calculate_rank_k_subspace_leverage_score(A, i, k)
    # Indices of columns we want to keep
    keep = list()
    levg_sum = 0
    # Get order of indices from highest to lowest
    order = np.argsort(-leverage_scores)
    ind = 0
    while(levg_sum <= k - epsilon):
        pi_i = order[ind]
        keep.append(pi_i)
        levg_sum += leverage_scores[pi_i]
        ind += 1
    while(len(keep) < k):
        pi_i = order[ind]
        keep.append(pi_i)
        levg_sum += leverage_scores[pi_i]
        ind += 1
    keep.sort()
    DCSS = A[:,keep]
    drop = list(order[ind::])
    return(DCSS, keep, drop)
    
        
    
    

In [481]:
red, fas, mem = reduceBitMatrix('82040.tsv')

k = 1

dcss1 = supposedDCSS(red.T, k, epsilon = 0.1)
print(dcss1[0].shape)
print(sum(red[dcss1[2],:]))


dcss2 = homemade_DCSS(red.T, k, epsilon = 0.1)
print(dcss2[0].shape)
print(red[dcss2[2],:])

(6, 20)
[4. 4. 3. 2. 4. 3.]
(6, 20)
[[0. 0. 1. 0. 1. 0.]
 [1. 0. 1. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0.]
 [1. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]


In [473]:
b = dict()

b['Alli'] = ["12"]
b['Nilli'] = ["0"]
b['Alli'].append("11")
    
(";").join(b['Nilli'])

'0'